In [1]:
################################ Load all dependencies we need
!pip install pygame
import numpy as np
import torch
from torch import nn, optim
import os 
from torch.utils.data import Dataset, DataLoader    
from PIL import Image
from tqdm import tqdm 
from efficientnet_pytorch import EfficientNet
import matplotlib.pyplot as plt
import cv2
from pygame import mixer

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
############################################ Hyperparameters 

def seed_everything(seed: int):
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

SEED = 42
DEVICE =  'cpu'
BATCH_SIZE = 128
LEARNING_RATE = 4e-4
NUM_EPOCH = 3
seed_everything(SEED)


In [3]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.eff    = EfficientNet.from_pretrained('efficientnet-b0')
        self.out    = nn.Linear(1000, 1)
        self.sigmoid= nn.Sigmoid()

    def forward(self, x):
        x = x.to(torch.float32)
        x = self.eff(x)
        #print(x.shape)
        x = self.out(x)
        x = self.sigmoid(x)
        return x
    
model = Model()

Loaded pretrained weights for efficientnet-b0


In [4]:
model.load_state_dict(torch.load("checkpoint.h5", map_location=torch.device('cpu')), model)
mixer.init()
model.eval()
sound = mixer.Sound("alarm.wav")
eyes_cascade = cv2.CascadeClassifier("haarcascade_eye.xml")
cap = cv2.VideoCapture(0)
score = 0
while True:
    try:
        _, img = cap.read()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        eyes = eyes_cascade.detectMultiScale(gray, 1.1, 4)
        for(x, y, w, h) in eyes:
            cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
            eye = img[x:x+w, y:y+h]
            eye = cv2.resize(eye, (97,97))
            eye = eye.reshape(3,97,97)
            eye = torch.from_numpy(eye).unsqueeze(0)
            pred = torch.sigmoid(model(eye))
            print(pred.item())
            # if eye is close 
            if pred.item() > 0.5:
                score = score + 1
                if score > 20:
                    score = 15
                if score >= 15:
                    sound.play()    
            # if eye is open:
            else:
                score = score -1 
                if score <0:
                    score = 0

        cv2.imshow('img', img)
    except:
        pass
    k = cv2.waitKey(30) & 0xff
    if k == 27 :
        break

cap.release()

0.6179691553115845
0.6531133651733398
0.6567111015319824
0.6112861037254333
0.5864836573600769
0.6774964332580566
0.6173434257507324
0.6706685423851013
0.6469748616218567
0.6040922403335571
0.6525307893753052
0.6299912333488464
0.607438862323761
0.6370693445205688
0.6416999101638794
0.683182418346405
0.637637734413147
0.6420544385910034
0.6414192914962769
0.6538057327270508
0.5857619643211365
0.5830491781234741
0.5897537469863892
0.6043668985366821
0.6513974666595459
0.6236905455589294
0.566434383392334
0.591951847076416
0.6128054261207581
0.6128313541412354
0.6352307796478271
0.6060872673988342
0.6050863265991211
0.5776905417442322
0.5897089242935181
0.5994642376899719
0.6590789556503296
0.6258326768875122
0.612572968006134
0.6517893671989441
0.6583791971206665
0.554699182510376
0.6237736344337463
0.6159033179283142
0.6086563467979431
0.6129919290542603
0.5819379687309265
0.5918964743614197
0.5768590569496155
0.665325403213501
0.624601423740387
0.640997588634491
0.6319777965545654
0.6